<div align="center">
<img src="https://user-images.githubusercontent.com/48846576/121096632-33afc800-c7b8-11eb-970a-0af7433de15a.png" alt="fastai" width="300" height="200"/> 
<p>In this notebook we will use fastai library to classify the <a href="http://https://www.kaggle.com/c/siim-covid19-detection">SIIM-FISABIO-RSNA COVID-19 Detection</a> images into the following categories</p>
<div align="left">    
<ul>
    <li>Negative </li>
    <li>Typical </li>
    <li>Atypical </li>
    <li>Indeterminate</li>
    </ul>
    </div>    
</div>

I'm using resized images from my other notebook https://www.kaggle.com/rajsengo/image-resize-siim-covid-19-detection using which I learnt the image resize techniques.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import gc

from colorama import Fore, Back, Style

y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
res = Style.RESET_ALL

import warnings
warnings.filterwarnings('ignore')

PATH = '/kaggle/input/siim-covid19-detection/'
submission = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv', index_col=None)
image_df = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv', index_col=None)
study_df = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv', index_col=None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)
print(f"{y_}Train image level csv shape : {image_df.shape}{res}\n{g_}Train study level csv shape : {study_df.shape}{res}")

In [ ]:
import os
all_files = []
for dirname, _, filenames in os.walk('/kaggle/input/image-resize-siim-covid-19-detection/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!mkdir train
!cd train
!tar xzf /kaggle/input/image-resize-siim-covid-19-detection/train.tar.gz

In [ ]:
!pwd

## Load Dataset

In [ ]:
from fastai.vision.data import ImageDataLoaders
from fastai.vision import models
from fastai.vision.all import cnn_learner
from fastai.metrics import RocAuc, accuracy, error_rate
from fastai.interpret import ClassificationInterpretation
from pathlib import Path

In [ ]:
trn_path = Path('/kaggle/working/')
dl = ImageDataLoaders.from_folder(trn_path, train='train', bs=8, valid_pct=.1, seed=42)
dl.train.show_batch()

In [ ]:
dl.valid.show_batch()

## Transfer Learning

Transfer learning is popular approach in deep learning where a model trained on a large dataset is reused for another classification task. We typically load a pre-trained model as a starting point instead of creating a model from scratch and retrain the model to adjust the weights for the specific dataset that we have.

The fastai library includes several pretrained models from torchvision, like resnet18, resnet34, resnet50, resnet101, resnet152, alexnet, etc. More details [here](https://fastai1.fast.ai/vision.models.html)

## ResNets

Residual networks are deep convolutional neural networks introduced by Microsoft paper ["Deep Residual Learning for Image Recognition"](https://arxiv.org/pdf/1512.03385.pdf). These CNNs use shortcut connections to skip one or more layers. ResNet addresses vanishing gradient problem which is a problem with gradient-based learning methods and backpropaga- tion where the gradient becomes very small after certain epochs, effectively preventing the weights from adjusting its value. ResNet50 has 50 layers and each resnet block is three layers deep.ResNet models are trained on ImageNet dataset

Here are the steps

- Load pre-trained resnet50
- Use 1 cycle policy and find learning rates. Detailed explanation is [here](https://iconof.com/1cycle-learning-rate-policy/) 
- Retrain with best learning rate
- Validate and Interpret results

In [ ]:
learn = cnn_learner(dl, models.resnet50, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.unfreeze()
lr_min, lr_steep = learn.lr_find()

In [ ]:
lr_min

In [ ]:
learn.fit_one_cycle(20, lr_min)

In [ ]:
learn.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(12, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(8,8), cmap='viridis_r', dpi=75)

In [ ]:
interp.most_confused()

In [ ]:
interp.print_classification_report()